https://dev.twitter.com/rest/reference/get/users/lookup

Script takes list of user_id s and returns the specified information for each user. Script reads user_id s from "user_lookup_in.txt". Before running the script specify your delimeter: ',' (comma), ' ' (space) or '\n' (new line) as default.

user_show is used to retrieve a single user object.

There are a few things to note when using this method:

* You must be following a protected user to be able to see their most recent status update. If you don’t follow a protected user their status will be removed.
* The order of user IDs or screen names may not match the order of users in the returned array.
* If a requested user is unknown, suspended, or deleted, then that user will not be returned in the results list.
* If none of your lookup criteria can be satisfied by returning a user object, a HTTP 404 will be thrown.

1. You need to enter your API credentials to the key list.
2. You can specify which info you need to the info list. You can find the name of the returned parameters (id, followers_count etc.) in the link above.
3. Script outputs the resulting data to user_lookup_out.txt as a csv formatted file

In [1]:
from birdy.twitter import UserClient, BirdyException 
from time import sleep
import string

key = [
    ["Consumer Key", "Consumer Secret", "Access Token", "Access Token Secret"]
]

client = UserClient(key[0][0], key[0][1], key[0][2], key[0][3])

In [40]:
delim = '\n'

f = open('user_lookup_in.txt', 'r', encoding='utf-8')
userIDs = f.read().split(delim)
f.close()
    
print('Number of user IDs: ' + str(len(userIDs)))

info = [
    'id',
    'screen_name',
    'description',
    'followers_count',
    'statuses_count'
]

Number of user IDs: 205


In [42]:
userInfo = []

for i in range(int(len(userIDs)/100)+1):
    imin = i*100
    imax = (i+1)*100
    if i == int(len(userIDs)/100):
        imax = len(userIDs)
    while(True):
        try:
            response = client.api.users.lookup.get(user_id=userIDs[imin : imax])
            for rd in response.data:
                tmpInfo = []
                for i in info:
                    tmpInfo.append(rd[i])
                userInfo.append(tmpInfo)
            break
        except Exception as err:
            print(err.status_code)
            print(err)
            if err.status_code == 429:
                sleep(60)
                keyInd = (keyInd + 1)%len(key)
            else:
                break
            client = UserClient(key[keyInd][0], key[keyInd][1], key[keyInd][2], key[keyInd][3])

In [44]:
filename = 'user_lookup_out.txt'
f = open(filename, 'w', encoding='utf-8')

f.write(str(info[0]))
for i in info[1:]:
    f.write(',')
    f.write(str(i))
f.write("\n")

for ui in userInfo:
    f.write(str(ui[0]))
    for i in ui[1:]:
        f.write(',')
        f.write(str(i))
    f.write("\n")
f.close()